<a href="https://colab.research.google.com/github/zhuanxuhit/cs20si/blob/master/03_logreg_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考 [TensorFlow全新的数据读取方式：Dataset API入门教程](https://zhuanlan.zhihu.com/p/30751039)

In [1]:
!git clone https://github.com/chiphuyen/stanford-tensorflow-tutorials.git tutorials

fatal: destination path 'tutorials' already exists and is not an empty directory.


In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# import utils

In [0]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000


In [0]:
mnist_folder = 'data/mnist'

In [4]:
!mkdir -pv data/mnist

mkdir: created directory 'data'
mkdir: created directory 'data/mnist'


In [0]:
import tutorials.examples.utils as utils

In [6]:
utils.download_mnist(mnist_folder)
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

Successfully downloaded data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded data/mnist/t10k-labels-idx1-ubyte.gz


In [7]:
len(train),len(train[0])

(2, 55000)

In [9]:
type(train)

tuple

此处重点讲解下 dataset的使用，先看一张图：
![继承图](https://pic2.zhimg.com/80/v2-f9f42cc5c00573f7baaa815795f1ce45_hd.jpg)

Dataset可以看作是相同类型“元素”的有序列表。在实际使用时，单个“元素”可以是向量，也可以是字符串、图片，甚至是tuple或者dict。

In [0]:
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # optional
test_data = tf.data.Dataset.from_tensor_slices(test)


利用tf.data.Dataset.from_tensor_slices创建每个元素是一个tuple的dataset也是可以的：

```
dataset = tf.data.Dataset.from_tensor_slices(
  (np.array([1.0, 2.0, 3.0, 4.0, 5.0]), np.random.uniform(size=(5, 2)))
)
```



In [0]:
train_data = train_data.batch(batch_size)
test_data = test_data.batch(batch_size)

##对Dataset中的元素做变换：Transformation

Dataset支持一类特殊的操作：Transformation。一个Dataset通过Transformation变成一个新的Dataset。通常我们可以通过Transformation完成数据变换，打乱，组成batch，生成epoch等一系列操作。

常用的Transformation有：

- map
- batch
- shuffle
- repeat


In [13]:
train_data.output_types

(tf.float32, tf.float64)

In [14]:
train_data.output_shapes

(TensorShape([Dimension(None), Dimension(784)]),
 TensorShape([Dimension(None), Dimension(10)]))

In [0]:
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
img, label = iterator.get_next()

In [0]:
train_init = iterator.make_initializer(train_data)	# initializer for train_data
test_init = iterator.make_initializer(test_data)	# initializer for test_data

In [0]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
w = tf.get_variable(name='weights', shape=(784, 10), initializer=tf.random_normal_initializer(0, 0.01))
b = tf.get_variable(name='bias', shape=(1, 10), initializer=tf.zeros_initializer())

In [0]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
logits = tf.matmul(img, w) + b 

In [0]:
# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=label, name='entropy')
loss = tf.reduce_mean(entropy, name='loss') # computes the mean over all the examples in the batch


In [0]:
# Step 6: define training op
# using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


In [0]:
# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

In [22]:
writer = tf.summary.FileWriter('./graphs/logreg', tf.get_default_graph())
with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0: 0.3649046092185863
Average loss epoch 1: 0.29023540718264357
Average loss epoch 2: 0.2826536399680515
Average loss epoch 3: 0.2731735230878342
Average loss epoch 4: 0.27216467266512473
Average loss epoch 5: 0.26859085571973823
Average loss epoch 6: 0.2663998494314593
Average loss epoch 7: 0.26637477374007534
Average loss epoch 8: 0.26438443808015005
Average loss epoch 9: 0.2578755683677141
Average loss epoch 10: 0.2590262358445068
Average loss epoch 11: 0.2608589655091596
Average loss epoch 12: 0.2559169008115003
Average loss epoch 13: 0.2549902643921763
Average loss epoch 14: 0.2544778812416764
Average loss epoch 15: 0.253373984264773
Average loss epoch 16: 0.25371954588349477
Average loss epoch 17: 0.2548387814573077
Average loss epoch 18: 0.2543502799646799
Average loss epoch 19: 0.25282337293028834
Average loss epoch 20: 0.25238093278089235
Average loss epoch 21: 0.2517545375713082
Average loss epoch 22: 0.2494045763688032
Average loss epoch 23: 0.252516897975